# Wstęp

Normalizacja i standaryzacja należą do najważniejszych operacji, jakie należy przeprowadzić przed przystąpieniem do prowadzenia eksperymentu uczenia maszynowego. Ze względu na fakt, że sieci neuronowe są aktywowane na podstawie podbudzenia (sumy wejść), jest bardzo ważne, żeby różnice w jedostkach nie były na tyle znaczące, by zaburzać proces. W ramach tego notebooka przyjrzymy się metodom normalizacji i standaryzacji - od postych do dużo bardziej złożonych.

In [ ]:
%pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
import gc
import os

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras_tuner as kt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
%matplotlib inline

# Wczytanie danych

W ramach tego ćwiczenia będziemy pracować na zbiorze danych **California Housing** opisującym ceny mieszkań w Kaliforni z lat 90-tych XX wieku. Dane zbierane były podczas spisu powszechnego i zostały pogrupowane według tzw. bloków, czyli podgrup o podobnym profilu. Zadaniem będzie przewidzenie mediany cen mieszkań dla określonych dzielnic.

Zbiór danych składa się z następujących cech, opisanych bardzo rożnymi cechami, o zmiennych zakresach wartości:


| cecha                  | opis                                                                                 | wart. min.               | wart. max.               |
|------------------------|--------------------------------------------------------------------------------------|--------------------------|--------------------------|
| **Longitude/latitude** | informacja o położeniu domu dalej  na wschód/zachód lub północ/południe.             | long.: -124.3 lat.: 32.5 | long.: -114.3 lat.: 42.5 |
| **HousingMedianAge**   | Mediana (w latach) wieku mieszkań wchodzących w skład badanego bloku                 | 1                        | 52                       |
| **TotalRooms**         | Całkowita liczba pokojów w badanym bloku                                             | 2                        | 37937                    |
| **TotalBedrooms**      | Całkowita liczba sypialni w badanym bloku                                            | 1                        | 6445                     |
| **population**         | Całkowita liczba osób w badanym bloku                                                | 3                        | 35682                    |
| **households**         | Ilość gospodarstw domowych w badanym bloku, tzn. grup ludzi zamieszkujących razem    | 1                        | 6082                     |
| **medianIncome**       | Mediana przychodu gospodarstwa domowego (w dziesiątkach tysięcy USD)                 | 0.5                      | 15                       |
| **medianHouseValue**   | Zmienna zależna/predykcyjna - mediana wartości domu w badanym bloku (mierzona w 100 tyś. USD) | 14999                    | 500001                   |

Zmienne nie są znormalizowane, wyrażono je w różnych jednostkach, zbiór zawiera także wartości odstające, tzn. znacznie wykraczające poza pożądany zakres wartości.

In [ ]:
from sklearn.datasets import fetch_california_housing

In [ ]:
california = fetch_california_housing()

In [ ]:
#??? Proszę wykonać opis danych

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [ ]:
X = pd.DataFrame(california.data, columns=california.feature_names)
y = california.target

In [ ]:
X.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


# Uczenie sieci nieznormalizowanej

Dla porównania, zbudujemy dwa modele na **oryginalnych** danych, aby zobaczyć skalę problemu i wpływu braku normalizacji danych.

<div class='alert alert-block alert-warning'>
    <b>Zadanie</b>: przeprowadźmy prosty ekspertyment:
    <ol>
        <li>Podziel oryginalne dane na train-test, (% train = 0.8), random_state=123</li>
        <li>Wyszkol model, który zawsze przewiduje średnią wartości w zbiorze (<code>DummyRegressor</code>) treningowym</li>
        <li>Wyszkol model sieci neuronowej z dwoma warstwami ukrytmi <code> (Dense:8, relu), (Dense:8, relu), (Dense:1, relu) </code> i przewidującą wynik. Wielkość batcha i il. epok dobierz dowolnie.</li>
        <li>Dla sieci neurnowej użyj metryk: <code>mean_absolute_error, mean_absolute_percentage_error</code></li>
        <li>Porównaj otrzymane wyniki na danych testowych używając metryk <code>MSE, MAPE</code></li>
    </ol>
</div>


In [ ]:
#???

In [ ]:
from sklearn.dummy import DummyRegressor
#???

DummyRegressor()

Sprawdzenie, czy wynik jest zgodny z oczekiwaniami:

In [ ]:
#???


MAPE dummy: 0.6177900742234355
MSE dummy: 1.3298460219228103


In [ ]:
#???

Epoch 1/5
516/516 [==============================] - 3s 4ms/step - loss: 5.6156 - mean_absolute_error: 2.0697 - mean_absolute_percentage_error: 100.0000
Epoch 2/5
516/516 [==============================] - 2s 4ms/step - loss: 5.6156 - mean_absolute_error: 2.0697 - mean_absolute_percentage_error: 100.0000
Epoch 3/5
516/516 [==============================] - 2s 4ms/step - loss: 5.6156 - mean_absolute_error: 2.0697 - mean_absolute_percentage_error: 100.0000
Epoch 4/5
516/516 [==============================] - 2s 4ms/step - loss: 5.6156 - mean_absolute_error: 2.0697 - mean_absolute_percentage_error: 100.0000
Epoch 5/5
516/516 [==============================] - 2s 3ms/step - loss: 5.6156 - mean_absolute_error: 2.0697 - mean_absolute_percentage_error: 100.0000


Sprawdzenie, czy wynik jest zgodny z oczekiwaniami:

In [ ]:
#???

129/129 [==============================] - 1s 4ms/step
MAPE: 1.0
MSE: 5.590082140508696


<div class='alert alert-block alert-info'>
Zapewne Twoje wyniki nie są zbyt zadowalające - bez nawet podstawowej normalizacji, sieć neuronowa powinna mieć około 100% błędu względnego. Tymczasem klasyfikator 'dummy' oscyluje w okolicach błędu na poziomie 61%. Nie jest dobrze.
</div>

# Skalowanie min-max

Sprawdzimy teraz wpływ skalowania min-max na wyniki uzyskiwane przez sieć neuronową.

<div class='alert alert-block alert-warning'>
    <b>Zadanie</b>: przeprowadźmy prosty ekspertyment:
    <ol>
        <li>Utwórz obiekty typu <code>MinMax Scaler</code> z pakietu Sklearn.</li>
        <li>Przetrenuj go na danych treningowych <code>X</code>, a następnie przetransformuj z jego użyciem dane treningowe i testowe, zapisując do osobnej zmiennej.</li>
        <li>Wyszkol model sieci neuronowej <b>o identycznzej architekutrze jak przedtem</b>, ale posługując się danymi przeskalowanymi techniką Min-Max.</li>
        <li>Dla sieci neurnowej użyj metryk: <code>mean_absolute_error, mean_absolute_percentage_error</code></li>
        <li>Porównaj otrzymane wyniki z poprzednią siecią i regresorem do średniej, na przeskalowanych danych testowych używając metryk <code>MSE, MAPE</code></li>
    </ol>
</div>

In [ ]:
#???

Epoch 1/5
516/516 [==============================] - 4s 5ms/step - loss: 1.8286 - mean_absolute_error: 1.0254 - mean_absolute_percentage_error: 59.0647
Epoch 2/5
516/516 [==============================] - 3s 5ms/step - loss: 0.6868 - mean_absolute_error: 0.6303 - mean_absolute_percentage_error: 39.9324
Epoch 3/5
516/516 [==============================] - 2s 4ms/step - loss: 0.5803 - mean_absolute_error: 0.5614 - mean_absolute_percentage_error: 33.3015
Epoch 4/5
516/516 [==============================] - 2s 4ms/step - loss: 0.5639 - mean_absolute_error: 0.5512 - mean_absolute_percentage_error: 32.3357
Epoch 5/5
516/516 [==============================] - 1s 2ms/step - loss: 0.5508 - mean_absolute_error: 0.5438 - mean_absolute_percentage_error: 31.8156


Sprawdzenie, czy wynik jest zgodny z oczekiwaniami:

In [ ]:
yhat_model2 = model2.predict(minmax.transform(X_test))

mape_model2 = mean_absolute_percentage_error(y_test, yhat_model2)
mse_model2 = mean_squared_error(y_test, yhat_model2)

print(f"MAPE model2: {mape_model2}")
print(f"MSE mdoel2: {mse_model2}")

assert 0.25 <= mape_model2 <= 0.45

129/129 [==============================] - 0s 2ms/step
MAPE model2: 0.32034155210231635
MSE mdoel2: 0.5317505017608231


<div class='alert alert-block alert-info'>
Uzyskane wyniki powinny się znacząco poprawić - spadek ze 100% błędu [%] do około 25-45% jest bardzo dużą poprawą.
</div>

# Standaryzacja danych

Wcześniej wykorzystywaliśmy skalowanie min-max. Teraz wykorzystamy typową

1.   Element listy
2.   Element listy

standaryzację danych.

<div class='alert alert-block alert-warning'>
    <b>Zadanie</b>: przeprowadź prosty ekspertyment:
    <ol>
        <li>Utwórz obiekty typu <code>StandardScaler</code> z pakietu Sklearn.</li>
        <li>Przetrenuj go na danych treningowych <code>X</code>, a następnie przetransformuj z jego użyciem dane treningowe i testowe, zapisując do osobnej zmiennej.</li>
        <li>Wyszkol model sieci neuronowej <b>o identycznzej architekutrze jak przedtem</b>, ale posługując się danymi przeskalowanymi techniką Standaryzacji.</li>
        <li>Dla sieci neurnowej użyj metryk: <code>mean_absolute_error, mean_absolute_percentage_error</code></li>
        <li>Porównaj otrzymane wyniki z poprzednią siecią i regresorem do średniej, na przeskalowanych danych testowych używając metryk <code>MSE, MAPE</code></li>
    </ol>
</div>

In [ ]:
#???

Epoch 1/5
516/516 [==============================] - 2s 2ms/step - loss: 1.7897 - mean_absolute_error: 0.9765 - mean_absolute_percentage_error: 51.6503
Epoch 2/5
516/516 [==============================] - 2s 3ms/step - loss: 0.6231 - mean_absolute_error: 0.5721 - mean_absolute_percentage_error: 35.4465
Epoch 3/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4800 - mean_absolute_error: 0.5020 - mean_absolute_percentage_error: 30.5564
Epoch 4/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4372 - mean_absolute_error: 0.4765 - mean_absolute_percentage_error: 28.3444
Epoch 5/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4256 - mean_absolute_error: 0.4684 - mean_absolute_percentage_error: 27.4767


Sprawdzenie, czy wynik jest zgodny z oczekiwaniami:

In [ ]:
yhat_model3 = model3.predict(sscaler.transform(X_test))

mape_model3 = mean_absolute_percentage_error(y_test, yhat_model3)
mse_model3 = mean_squared_error(y_test, yhat_model3)

print(f"MAPE model3: {mape_model3}")
print(f"MSE model3: {mse_model3}")

assert 0.2 <= mape_model3 <= 0.35

129/129 [==============================] - 0s 2ms/step
MAPE model3: 0.27119682301132175
MSE model3: 0.40499974028669555


<div class='alert alert-block alert-info'>
Uzyskane wyniki powinny się znacząco poprawić w stosunku do początkowego modelu - spadek ze 100% błędu [%] do około 20-35% jest bardzo dużą poprawą. Jendocześnie, różnica pomiędzy normalizacją a standardyzacją nie powinna być zbyt zauważalna - oba modele powinny sprawować się bardzo podobnie.
</div>

# Batch Norm

W tej części notebooka wykorzystamy technikę, która pozwala sieci neuronowej przeprowadzić normalizację danych "według uznania", tzn. w taki sposób, który będzie najlepiej poprawiał uzyskiwane przez nią wyniki. Uzyskamy to dzięki dodaniu nowej warstwy: 'batch norm' na samym początku sieci.

<div class='alert alert-block alert-warning'>
    <b>Zadanie</b>: przeprowadź prosty ekspertyment:
    <ol>
        <li>Wyszkol model sieci neuronowej, którego pierwsza wastwa to <code>BatchNormalization</code>, która pozwoli odpowiednio dostosować dane  </li>
        <li>Reszta architektury sieci (kolejne warstwy i aktywacje) powinna być <b>identyczna jak przedtem</b></li>
        <li>Dla sieci neurnowej użyj metryk: <code>mean_absolute_error, mean_absolute_percentage_error</code></li>
        <li>Porównaj otrzymane wyniki z poprzednią siecią i regresorem do średniej, na przeskalowanych danych testowych używając metryk <code>MSE, MAPE</code></li>
    </ol>
</div>

In [ ]:
#???

Epoch 1/5
516/516 [==============================] - 2s 2ms/step - loss: 0.9530 - mean_absolute_error: 0.7149 - mean_absolute_percentage_error: 40.9243
Epoch 2/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4969 - mean_absolute_error: 0.5211 - mean_absolute_percentage_error: 31.0327
Epoch 3/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4641 - mean_absolute_error: 0.5001 - mean_absolute_percentage_error: 29.8024
Epoch 4/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4620 - mean_absolute_error: 0.4988 - mean_absolute_percentage_error: 29.4495
Epoch 5/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4488 - mean_absolute_error: 0.4865 - mean_absolute_percentage_error: 28.5784


In [ ]:
#???

assert 0.2 <= mape_model4 <= 0.35

129/129 [==============================] - 0s 1ms/step
MAPE model4: 0.24743245026317312
MSE model4: 0.4882442248505259


<div class='alert alert-block alert-info'>
Uzyskane wyniki powinny się znacząco poprawić w stosunku do początkowego modelu - spadek ze 100% błędu [%] do około 20-35% jest bardzo dużą poprawą. Jendocześnie, różnica pomiędzy normalizacją, standardyzacją a użyciem techniki <b>batch normalization </b> nie powinna być zbyt zauważalna - wszystkie trzy modele powinny się sprawować w podobny sposób.
</div>

# To wszystko?

Sprawdźmy we własnym zakresie, jak zmiana architektury sieci, zmiana wielkości batcha lub **ilości** warstw BatchNormalization (przeplatanych warstwami Dense) wpłynie na uzyskany wynik. W kolejnych notebookach będziemy się zajmować dokładną analizą, czy uzyskane wyniki są **statystycznie istotnie lepsze**, jednak na raize można to sprawdzić "naocznie" przez porównanie uzyskanych statystyk.

Warto w tym miejscu zaznaczyć, że przy wykorzystaniu **Batch normalization** nie ma raczej sensu używanie innych procedur normalizacji/standdardyzacji, ponieważ sieć neuronowa "zajmie się" tym wewnętrznie w procesie uczenia.

<div class='alert alert-block alert-warning'>
    <b>Zadanie</b>: przeprowadźmy kompleksowy eksperyment szukając najlepszej architektury sieci oraz sposobu normalizacji danych - Min-max, standaryzacja lub <b>batch normalization</b>.
    Możemy wykorzystywać Keras Tuner lub "ręczną" walidację krzyżową - według uznania. Jeśli samodizelnie chcesz prowadzić wybór architeltury, wówczas:
    <ol>
        <li>"Opakowauj" różne architektury modeli za pomocą <code>tf.keras.wrappers.scikit_learn.KerasRegressor</code></li>
        <li>Prowadź 10-o krotną walidację krzyżową na zbiorze treningowym, przyjmując jako metrykę MSE</li>
        <li>Na bieząco porónuj otrzymywane wyniki z rezultatami na zbiorze testowym</li>
        <li>Na koniec - gdy na podstawie eksperymentów wybierzesz najlepszy wariant,  przeszkol wybraną architekturę sieci na <b>całym zbiorze treningowym</b>, zbierz metryki MSE i MAPE i porównaj ze zbiorem testowym</li>
    </ol>
</div>

## Ręczny tuning

In [ ]:
#???

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

sklearn_wrap = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model, epochs=5, batch_size=16, verbose=0)
cv_scores = cross_val_score(sklearn_wrap, X_train, y_train, cv=10, scoring=make_scorer(mean_squared_error))

<ipython-input-21-ecd46bd07077>:4: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  sklearn_wrap = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model, epochs=5, batch_size=16, verbose=0)


In [ ]:
np.mean(cv_scores)

0.9697879603539429

In [ ]:
final_model = build_model()
final_model.fit(X_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
516/516 [==============================] - 2s 2ms/step - loss: 1.2202 - mean_absolute_error: 0.7965 - mean_absolute_percentage_error: 43.6300
Epoch 2/5
516/516 [==============================] - 1s 2ms/step - loss: 0.5039 - mean_absolute_error: 0.5225 - mean_absolute_percentage_error: 31.0008
Epoch 3/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4612 - mean_absolute_error: 0.4974 - mean_absolute_percentage_error: 29.2483
Epoch 4/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4541 - mean_absolute_error: 0.4906 - mean_absolute_percentage_error: 28.7105
Epoch 5/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4457 - mean_absolute_error: 0.4852 - mean_absolute_percentage_error: 28.3050


In [ ]:
yhat_final = final_model.predict(X_test)

print(f"MAPE final: {mean_absolute_percentage_error(y_test, yhat_final)}")
print(f"MSE final: {mean_squared_error(y_test, yhat_final)}")

129/129 [==============================] - 0s 2ms/step
MAPE final: 0.2710390645212285
MSE final: 0.5080479103305098


## Keras tuner

In [ ]:
def complex_architecture_search(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization())

    hp_nlayers = hp.Int('nlayers', min_value=1, max_value=4, step=1)
    for l_idx in range(hp_nlayers):
        hp_units = hp.Int(f'units_layer_{l_idx}', min_value=4, max_value=32, step=2)
        hp_activations = hp.Choice(f'activation_layer_{l_idx}', values=['relu', 'tanh'])
        model.add(tf.keras.layers.Dense(hp_units, hp_activations, input_shape=(X_train.shape[1],)))
    model.add(tf.keras.layers.Dense(1, 'relu'))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.mean_absolute_error, tf.keras.metrics.mean_absolute_percentage_error])
    return model

In [ ]:
tuner = kt.Hyperband(complex_architecture_search,
                     objective='val_loss',
                     max_epochs=10,
                     hyperband_iterations=5,
                     directory=os.path.normpath(os.path.join("/content/", "keras_tuner")),
                     project_name='tuner_regressor')

In [ ]:
tuner.search(
    X_train,
    y_train,
    epochs=10,
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

Trial 150 Complete [00h 00m 22s]
val_loss: 0.3846227526664734

Best val_loss So Far: 0.3545277416706085
Total elapsed time: 00h 20m 04s


In [ ]:
best_params = tuner.get_best_hyperparameters()[0]
for i in range(best_params.get('nlayers')):
    print(f"Layer {i}, activation:", best_params.get(f'activation_layer_{i}'), "units: ", best_params.get(f'units_layer_{i}'))

Layer 0, activation: tanh units:  22
Layer 1, activation: relu units:  20
Layer 2, activation: tanh units:  26
Layer 3, activation: relu units:  30


In [ ]:
def tuner_builder():
    model = tuner.hypermodel.build(best_params)
    return model

In [ ]:
tuner_model = tuner_builder()
tuner_model.fit(X_train, y_train, epochs=5, batch_size=16)

Epoch 1/5
1032/1032 [==============================] - 3s 2ms/step - loss: 0.6896 - mean_absolute_error: 0.5984 - mean_absolute_percentage_error: 34.7832
Epoch 2/5
1032/1032 [==============================] - 3s 2ms/step - loss: 0.4898 - mean_absolute_error: 0.5119 - mean_absolute_percentage_error: 29.5944
Epoch 3/5
1032/1032 [==============================] - 3s 3ms/step - loss: 0.4718 - mean_absolute_error: 0.4971 - mean_absolute_percentage_error: 28.4978
Epoch 4/5
1032/1032 [==============================] - 3s 3ms/step - loss: 0.4727 - mean_absolute_error: 0.4985 - mean_absolute_percentage_error: 28.4532
Epoch 5/5
1032/1032 [==============================] - 3s 3ms/step - loss: 0.4667 - mean_absolute_error: 0.4965 - mean_absolute_percentage_error: 28.3575


In [ ]:
yhat_tuner = tuner_model.predict(X_test)
print(f"MAPE final: {mean_absolute_percentage_error(y_test, yhat_tuner)}")
print(f"MSE final: {mean_squared_error(y_test, yhat_tuner)}")

129/129 [==============================] - 0s 2ms/step
MAPE final: 0.2221618037266241
MSE final: 0.3484366660158147
